<a href="https://colab.research.google.com/github/EastHuni/lg-aimers/blob/main/%5BBaseline%5D_LSTM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EB%A9%94%EB%89%B4%EB%B3%84_%EB%A7%A4%EC%B6%9C_%EC%88%98%EB%9F%89_%EC%98%88%EC%B8%A1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm


# Fixed RandomSeed & Setting Hyperparameter

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data Load

In [ ]:
import zipfile
import os

zip_path = '/content/drive/MyDrive/LG/open.zip'
extract_path = '/content/LG_data'
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
train = pd.read_csv('/content/LG_data/train/train.csv')
train

,영업일자,영업장명_메뉴명,매출수량
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0
...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12
102672,2024-06-12,화담숲카페_현미뻥스크림,10
102673,2024-06-13,화담숲카페_현미뻥스크림,14
102674,2024-06-14,화담숲카페_현미뻥스크림,12


# Define Model

In [ ]:
pip install holidays

In [ ]:
import holidays
train['영업일자'] = pd.to_datetime(train['영업일자'])

kr_holidays = holidays.KR(years=[2023, 2024])

train['요일'] = train['영업일자'].dt.dayofweek #0 = 월요일, ..., 6 = 일요일
train['공휴일'] = train['영업일자'].isin(kr_holidays).astype(int)
train

/tmp/ipython-input-1359051083.py:13: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  train['공휴일'] = train['영업일자'].isin(kr_holidays).astype(int)


,영업일자,영업장명_메뉴명,매출수량,공휴일,요일
0,2023-01-01,느티나무 셀프BBQ_1인 수저세트,0,1,6
1,2023-01-02,느티나무 셀프BBQ_1인 수저세트,0,0,0
2,2023-01-03,느티나무 셀프BBQ_1인 수저세트,0,0,1
3,2023-01-04,느티나무 셀프BBQ_1인 수저세트,0,0,2
4,2023-01-05,느티나무 셀프BBQ_1인 수저세트,0,0,3
...,...,...,...,...,...
102671,2024-06-11,화담숲카페_현미뻥스크림,12,0,1
102672,2024-06-12,화담숲카페_현미뻥스크림,10,0,2
102673,2024-06-13,화담숲카페_현미뻥스크림,14,0,3
102674,2024-06-14,화담숲카페_현미뻥스크림,12,0,4


In [ ]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)


def train_lstm(train_df):
    trained_models = {}

    for store_menu, group in tqdm(train_df.groupby(['영업장명_메뉴명']), desc ='Training LSTM'):
        store_train = group.sort_values('영업일자').copy()
        if len(store_train) < LOOKBACK + PREDICT:
            continue

        features = ['매출수량','요일', '공휴일']
        scaler = MinMaxScaler()
        store_train[features] = scaler.fit_transform(store_train[features])
        train_vals = store_train[features].values  # shape: (N, 1)

        # 시퀀스 구성
        X_train, y_train = [], []
        for i in range(len(train_vals) - LOOKBACK - PREDICT + 1):
            X_train.append(train_vals[i:i+LOOKBACK])
            y_train.append(train_vals[i+LOOKBACK:i+LOOKBACK+PREDICT, 0])

        X_train = torch.tensor(X_train).float().to(DEVICE)
        y_train = torch.tensor(y_train).float().to(DEVICE)

        model = MultiOutputLSTM(input_dim=3, output_dim=PREDICT).to(DEVICE)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.MSELoss()

        model.train()
        for epoch in range(EPOCHS):
            idx = torch.randperm(len(X_train))
            for i in range(0, len(X_train), BATCH_SIZE):
                batch_idx = idx[i:i+BATCH_SIZE]
                X_batch, y_batch = X_train[batch_idx], y_train[batch_idx]
                output = model(X_batch)
                loss = criterion(output, y_batch)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        trained_models[store_menu] = {
            'model': model.eval(),
            'scaler': scaler,
            'last_sequence': train_vals[-LOOKBACK:]  # (28, 1)
        }

    return trained_models
# 학습
trained_models = train_lstm(train)
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted[['매출수량', '요일', '공휴일']].values[-LOOKBACK:]  # shape: (28, 3)
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals)
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 3))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val

            })

    return pd.DataFrame(results)

Training LSTM: 100%|██████████| 193/193 [55:39<00:00, 17.31s/it]
/tmp/ipython-input-2915933670.py:117: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_df['공휴일'] = test_df['영업일자'].isin(kr_holidays).astype(int)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,3)

# Train

# Prediction

In [ ]:
def predict_lstm(test_df, trained_models, test_prefix: str):
    results = []

    for store_menu, store_test in test_df.groupby(['영업장명_메뉴명']):
        key = store_menu
        if key not in trained_models:
            continue

        model = trained_models[key]['model']
        scaler = trained_models[key]['scaler']

        store_test_sorted = store_test.sort_values('영업일자')
        recent_vals = store_test_sorted[['매출수량', '요일', '공휴일']].values[-LOOKBACK:]  # shape: (28, 3)
        if len(recent_vals) < LOOKBACK:
            continue

        # 정규화
        recent_vals = scaler.transform(recent_vals)
        x_input = torch.tensor([recent_vals]).float().to(DEVICE)

        with torch.no_grad():
            pred_scaled = model(x_input).squeeze().cpu().numpy()

        # 역변환
        restored = []
        for i in range(PREDICT):
            dummy = np.zeros((1, 3))
            dummy[0, 0] = pred_scaled[i]
            restored_val = scaler.inverse_transform(dummy)[0, 0]
            restored.append(max(restored_val, 0))

        # 예측일자: TEST_00+1일 ~ TEST_00+7일
        pred_dates = [f"{test_prefix}+{i+1}일" for i in range(PREDICT)]

        for d, val in zip(pred_dates, restored):
            results.append({
                '영업일자': d,
                '영업장명_메뉴명': store_menu,
                '매출수량': val

            })

    return pd.DataFrame(results)

In [ ]:
all_preds = []

# 모든 test_*.csv 순회
test_files = sorted(glob.glob('/content/LG_data/test/TEST_*.csv'))

for path in test_files:
    test_df = pd.read_csv(path)
    test_df['영업일자'] = pd.to_datetime(test_df['영업일자'])

    # 공휴일 여부 컬럼 추가
    test_df['공휴일'] = test_df.index.to_series().isin(kr_holidays).astype(int)


    # 요일 / 공휴일 파생변수
    test_df['요일'] = test_df['영업일자'].dt.dayofweek
    test_df['공휴일'] = test_df['영업일자'].isin(kr_holidays).astype(int)


    # 파일명에서 접두어 추출 (예: TEST_00)
    filename = os.path.basename(path)
    test_prefix = re.search(r'(TEST_\d+)', filename).group(1)

    pred_df = predict_lstm(test_df, trained_models, test_prefix)
    all_preds.append(pred_df)

full_pred_df = pd.concat(all_preds, ignore_index=True)

/tmp/ipython-input-1867809092.py:16: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  test_df['공휴일'] = test_df['영업일자'].isin(kr_holidays).astype(int)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarn

In [ ]:
def convert_to_submission_format(pred_df: pd.DataFrame, sample_submission: pd.DataFrame):
    # (영업일자, 메뉴) → 매출수량 딕셔너리로 변환
    pred_dict = dict(zip(
      zip(pred_df['영업일자'], pred_df['영업장명_메뉴명'].apply(lambda x: x[0] if isinstance(x, tuple) else x)),
      pred_df['매출수량']
      ))


    final_df = sample_submission.copy()

    for row_idx in final_df.index:
        date = final_df.loc[row_idx, '영업일자']
        for cols in final_df.columns[1:]:  # 메뉴명들
            final_df.loc[row_idx, cols] = pred_dict.get((date, cols,), 0)

    return final_df


In [ ]:
sample_submission = pd.read_csv('/content/LG_data/sample_submission.csv')
submission = convert_to_submission_format(full_pred_df, sample_submission)
submission.to_csv('baseline_submission.csv', index=False, encoding='utf-8-sig')


# Submission

NameError: name 'pred_dict' is not defined